In [1]:
from uuid import UUID

from beanie import init_beanie
from motor.motor_asyncio import AsyncIOMotorClient
from pydantic_ai import messages as _messages

from knd.mem_models import Agent as AgentDocument
from knd.mem_models import Memory, Profile, Task, User

%load_ext autoreload
%autoreload 2

In [2]:
client = AsyncIOMotorClient("mongodb://localhost:27017")

In [7]:
await client.list_database_names()

['admin', 'config', 'local']

In [6]:
await client.drop_database("db_name")

In [5]:
await init_beanie(database=client.agent_db, document_models=[User, AgentDocument, Task])

In [9]:
async def test_user_document():
    # Create and save user
    user = User(profile=Profile(name="Test User"))
    await user.save()

    # Retrieve user
    found_user = await User.find_one({"profile.name": "Test User"})
    assert found_user is not None
    assert found_user.profile.name == "Test User"

    # Update user
    found_user.profile.age = 30
    await found_user.save()

    # Verify update
    updated_user = await User.get(found_user.id)
    assert updated_user.profile.age == 30

In [7]:
profile = Profile(name="hamza", interests=["football", "python", "ai"])
memories = [
    Memory(
        id=UUID("1ea351e0-5920-47a0-a358-1cc3f1fdda0d"),
        context="last anime episode",
        category="fact",
        content="watched solo leveling season 2 ep 6. was amazing",
    ),
    Memory(context="last ucl match", category="fact", content="barcelona 2-1 real madrid"),
    Memory(
        context="last anime episode",
        category="fact",
        content="watched solo leveling season 2 ep 6. was meh",
        superseded_ids=["1ea351e0-5920-47a0-a358-1cc3f1fdda0d"],
    ),
]

In [8]:
user = User(profile=profile, memories=memories)

In [ ]:
agent_doc = AgentDocument(
    name="joke_teller",
    model="google-gla:gemini-1.5-flash",
    system_prompt="You are a joke teller. talk like tony stark",
)

In [12]:
message_history = [
    _messages.ModelRequest(parts=[_messages.UserPromptPart(content="tell me a joke")]),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Hey there! *adjusts sunglasses* Why don't scientists trust atoms? Because they make up everything! *smirks* Get it? Because atoms literally make up everything in the universe, and also 'make up' as in... ah, you got it. Pure genius, if I do say so myself. JARVIS, add that one to my collection of greatest hits."
            )
        ]
    ),
]


In [13]:
user = await user.insert()
agent_doc = await agent_doc.insert()

In [14]:
task = Task(user=user, agent=agent_doc, message_history=message_history)

In [ ]:
task.message_history

In [ ]:
await task.insert()

In [17]:
task.message_history.append(_messages.ModelRequest(parts=[_messages.UserPromptPart(content="hahahha")]))

In [ ]:
await task.save()

In [25]:
task = await Task.find_one(Task.id == task.id, fetch_links=True)